# Meta-Labeling for bet side and size

Implementation of Triple-barrier method for determining side and Meta-labeling for size of the bet. Meta-labeling is a technique introduced by Marco Lopez De Prado in Advances to Financial machine learning.

## Imports

In [161]:
%load_ext autoreload
%autoreload 2

# standard imports
from pathlib import PurePath, Path 
import sys
import time
from collections import OrderedDict as od 
import re 
import os 
import json 

# scientific stack
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import math
import mlfinlab as ml
import sklearn as sk

# visual tools and plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import Dataset

In [162]:
# read in and store raw tick data in pandas dataframe to be cleaned and transformed
raw_tick_data = pd.read_csv('tick_data.csv')
raw_tick_data['date_time'] = pd.to_datetime(raw_tick_data.date_time)
raw_tick_data.set_index('date_time', drop=True, inplace=True)
print(raw_tick_data)

                            open     high      low    close  cum_vol  \
date_time                                                              
2011-07-31 23:31:58.810  1306.00  1308.75  1301.75  1305.75    53658   
2011-08-01 02:55:17.443  1305.75  1309.50  1304.00  1306.50    53552   
2011-08-01 07:25:56.319  1306.75  1309.75  1304.75  1305.00    53543   
2011-08-01 08:33:10.903  1305.00  1305.00  1299.00  1300.00    53830   
2011-08-01 10:51:41.842  1300.00  1307.75  1299.00  1307.75    53734   
...                          ...      ...      ...      ...      ...   
2012-07-30 12:30:28.642  1379.25  1380.00  1377.50  1377.75    50843   
2012-07-30 13:29:21.258  1377.75  1380.00  1377.00  1379.25    50782   
2012-07-30 13:35:05.407  1379.25  1383.25  1379.00  1382.50    50675   
2012-07-30 13:43:43.711  1382.50  1383.25  1380.00  1381.00    50667   
2012-07-30 13:54:26.158  1380.75  1381.75  1379.75  1380.75    50698   

                          cum_dollar  cum_ticks  
date_time    

## Create Dollar Bars

Here we transform the raw ohlc data into various financial data structures that provide a better representation of the movement of trade information throughout the market.  These bars include: dollar, tick, and volume bars.

## Fitting the Primary Model

This is is the primary model that we will use to drive our strategy (EMA Crossover)

In [163]:
fast_window = 12
slow_window = 26

raw_tick_data['fast_mavg'] = raw_tick_data['close'].rolling(window=fast_window, min_periods=fast_window, center=False).mean()
raw_tick_data['slow_mavg'] = raw_tick_data['close'].rolling(window=slow_window, min_periods=slow_window, center=False).mean()

### Compute side

In [164]:
raw_tick_data['side'] = np.nan
long_signals = raw_tick_data['fast_mavg'] >= raw_tick_data['slow_mavg']
short_signals = raw_tick_data['fast_mavg'] < raw_tick_data['slow_mavg']
raw_tick_data.loc[long_signals, 'side'] = 1
raw_tick_data.loc[short_signals, 'side'] = -1
data = raw_tick_data.dropna()
raw_data = data.copy()
print(long_signals)

date_time
2011-07-31 23:31:58.810    False
2011-08-01 02:55:17.443    False
2011-08-01 07:25:56.319    False
2011-08-01 08:33:10.903    False
2011-08-01 10:51:41.842    False
                           ...  
2012-07-30 12:30:28.642     True
2012-07-30 13:29:21.258    False
2012-07-30 13:35:05.407    False
2012-07-30 13:43:43.711    False
2012-07-30 13:54:26.158    False
Length: 10000, dtype: bool


In [165]:
# set daily volatility
daily_vol = ml.util.get_daily_vol(close=data['close'], lookback=50)
print(daily_vol)

date_time
2011-08-02 14:49:14.665         NaN
2011-08-02 14:55:22.761    0.000009
2011-08-02 15:02:12.277    0.000013
2011-08-02 15:07:17.932    0.001585
2011-08-02 15:12:42.641    0.001509
                             ...   
2012-07-30 12:30:28.642    0.008794
2012-07-30 13:29:21.258    0.009193
2012-07-30 13:35:05.407    0.009381
2012-07-30 13:43:43.711    0.009588
2012-07-30 13:54:26.158    0.009764
Name: close, Length: 9926, dtype: float64


In [166]:
# convert from daily vol to hourly vol (since our data in hourly)
trading_hours_in_day = 8
trading_days_in_year = 252
hourly_vol = daily_vol / math.sqrt(trading_hours_in_day * trading_days_in_year)
hourly_vol_mean = hourly_vol.mean()
print(hourly_vol)

date_time
2011-08-02 14:49:14.665             NaN
2011-08-02 14:55:22.761    2.095284e-07
2011-08-02 15:02:12.277    2.818900e-07
2011-08-02 15:07:17.932    3.530381e-05
2011-08-02 15:12:42.641    3.359834e-05
                               ...     
2012-07-30 12:30:28.642    1.958661e-04
2012-07-30 13:29:21.258    2.047418e-04
2012-07-30 13:35:05.407    2.089223e-04
2012-07-30 13:43:43.711    2.135460e-04
2012-07-30 13:54:26.158    2.174610e-04
Name: close, Length: 9926, dtype: float64


### Apply CUSUM Filter

The purpose of the CUSUM filter is to locate instances of change detection.  These are points that will later be used as t_events for the Triple-barrier method.

In [167]:
# apply symetric CUSUM filter and get timestamps for events
cusum_events = ml.filters.cusum_filter(data['close'], threshold=hourly_vol_mean * 0.5)
cusum_events

DatetimeIndex(['2011-08-01 14:54:45.324000', '2011-08-01 15:01:17.292000',
               '2011-08-01 15:05:30.877000', '2011-08-01 15:11:41.236000',
               '2011-08-01 15:26:40.200000', '2011-08-01 15:34:37.446000',
               '2011-08-01 15:38:23.090000', '2011-08-01 15:56:25.240000',
               '2011-08-01 16:16:58.143000', '2011-08-01 16:29:55.276000',
               ...
               '2012-07-27 20:01:25.794000', '2012-07-27 20:14:35.480000',
               '2012-07-30 06:13:28.136000', '2012-07-30 08:10:13.266000',
               '2012-07-30 10:16:04.616000', '2012-07-30 12:30:28.642000',
               '2012-07-30 13:29:21.258000', '2012-07-30 13:35:05.407000',
               '2012-07-30 13:43:43.711000', '2012-07-30 13:54:26.158000'],
              dtype='datetime64[ns]', length=9523, freq=None)

In [168]:
# compute vertical barrier
vertical_barriers = ml.labeling.add_vertical_barrier(t_events=cusum_events, close=data['close'], num_days=1)
vertical_barriers

2011-08-01 14:54:45.324   2011-08-02 14:55:22.761
2011-08-01 15:01:17.292   2011-08-02 15:02:12.277
2011-08-01 15:05:30.877   2011-08-02 15:07:17.932
2011-08-01 15:11:41.236   2011-08-02 15:12:42.641
2011-08-01 15:26:40.200   2011-08-02 15:27:24.004
                                    ...          
2012-07-27 19:53:34.640   2012-07-30 06:13:28.136
2012-07-27 19:57:45.879   2012-07-30 06:13:28.136
2012-07-27 19:59:58.054   2012-07-30 06:13:28.136
2012-07-27 20:01:25.794   2012-07-30 06:13:28.136
2012-07-27 20:14:35.480   2012-07-30 06:13:28.136
Name: date_time, Length: 9515, dtype: datetime64[ns]

### Set Take Profit and Stop Loss

In [169]:
pt_sl = [1,2]
min_ret = 0.005

### Get Triple-barrier Events using CUSUM filter

In [170]:
triple_barrier_events = ml.labeling.get_events(close=data['close'],
                                                t_events=cusum_events,
                                                pt_sl=pt_sl,
                                                #check between hourly and daily vol
                                                target=daily_vol,
                                                min_ret=min_ret,
                                                num_threads=3,
                                                vertical_barrier_times=vertical_barriers,
                                                side_prediction=data['side'],
                                                verbose=True
                                                )

labels = ml.labeling.get_bins(triple_barrier_events, data['close'])
labels

2020-10-07 15:06:07.878537 100.0% apply_pt_sl_on_t1 done after 0.13 minutes. Remaining 0.0 minutes..


,ret,trgt,bin,side
2011-08-02 19:31:14.387,0.005400,0.005069,1,-1.0
2011-08-02 19:42:30.586,0.006422,0.006108,1,-1.0
2011-08-02 19:45:56.176,0.007222,0.006424,1,-1.0
2011-08-02 19:50:37.185,0.009869,0.006789,1,-1.0
2011-08-02 19:52:57.011,0.009063,0.007127,1,-1.0
...,...,...,...,...
2012-07-27 19:53:34.640,-0.001991,0.005877,0,1.0
2012-07-27 19:57:45.879,-0.001268,0.005763,0,1.0
2012-07-27 19:59:58.054,-0.001810,0.005655,0,1.0
2012-07-27 20:01:25.794,-0.002713,0.005590,0,1.0


## Results of the Primary Model

In [171]:
# set variables for analysis
primary_forecast = pd.DataFrame(labels['bin'])
primary_forecast['pred'] = 1
primary_forecast.columns = ['actual', 'pred']

In [172]:
# performance metrics
actual = primary_forecast['actual']
pred = primary_forecast['pred']

#### Classification Report

In [173]:
print('CLassification Report')
print(sk.metrics.classification_report(y_true=actual,  y_pred=pred, zero_division=False))

CLassification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4659
           1       0.40      1.00      0.58      3162

    accuracy                           0.40      7821
   macro avg       0.20      0.50      0.29      7821
weighted avg       0.16      0.40      0.23      7821



#### Confusion Matrix

In [174]:
print('Confusion Matrix')
sk.metrics.confusion_matrix(actual, pred)

Confusion Matrix


array([[   0, 4659],
       [   0, 3162]], dtype=int64)

#### Accuracy

In [175]:
print('Accuracy')
sk.metrics.accuracy_score(actual, pred)

Accuracy


0.40429612581511315

## Fit the Meta Model (Train Forest for bet size)

In [176]:
# get log of returns
raw_data['log_ret'] = np.log(raw_data['close']).diff()

In [177]:
# create momentum columns
raw_data['mom1'] = raw_data['close'].pct_change(periods=1)
raw_data['mom2'] = raw_data['close'].pct_change(periods=2)
raw_data['mom3'] = raw_data['close'].pct_change(periods=3)
raw_data['mom4'] = raw_data['close'].pct_change(periods=4)
raw_data['mom5'] = raw_data['close'].pct_change(periods=5)

In [178]:
# create volatility columns
raw_data['volatility_50'] = raw_data['log_ret'].rolling(window=50, min_periods=50, center=False).std()
raw_data['volatility_31'] = raw_data['log_ret'].rolling(window=31, min_periods=31, center=False).std()
raw_data['volatility_15'] = raw_data['log_ret'].rolling(window=15, min_periods=15, center=False).std()

In [ ]:
# serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_2'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
raw_data['autocorr_4'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
raw_data['autocorr_5'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

In [ ]:
# serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_2'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
raw_data['autocorr_4'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
raw_data['autocorr_5'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

### Recompute sides

In [ ]:
raw_data['side'] = np.nan
long_signals = raw_data['fast_mavg'] >= raw_data['slow_mavg']
short_signals = raw_data['fast_mavg'] < raw_data['slow_mavg']
raw_data.loc[long_signals, 'side'] = 1
raw_data.loc[short_signals, 'side'] = -1

In [ ]:
long_signals

In [ ]:
# remove look ahead bias
raw_data = raw_data.shift(1)

In [ ]:
# get features at event times
x = raw_data.loc[labels.index, :]
x.head()

In [ ]:
# remove unwanted columns
x.drop(['open', 'high', 'low', 'close', 'cum_vol', 'cum_dollar', 'cum_ticks','fast_mavg', 'slow_mavg'], axis=1, inplace=True)
y = labels['bin']
x.head()

In [ ]:
# print the y values (0, 1)
y.value_counts()

### Model Training

Here we will strart by seperating our dataset in a training and test set for the purpose of implementing our random forest and checking itts abiility to accurately make predictions on the test set.

#### Split into training and test set

In [ ]:
raw_data = raw_data.dropna()
x_training_validation = x
y_training_validation = y

In [ ]:
# function to create train, test, and split data from scikit-learn
x_train, x_validate, y_train, y_validate = sk.model_selection.train_test_split(
                                        x_training_validation, 
                                        y_training_validation,
                                        test_size=0.70,
                                        train_size=0.30,  
                                        shuffle=False)

In [ ]:
# create train dataframe
train_df = pd.concat([y_train,x_train], axis=1, join='inner')
train_df['bin'].value_counts()

#### Unsample training data for 50/50 split

In [ ]:
majority = train_df[train_df['bin'] == 0]
minority = train_df[train_df['bin'] == 1]

In [ ]:
majority = train_df[train_df['bin'] == 0]
minority = train_df[train_df['bin'] == 1]

new_minority = sk.utils.resample(minority,
                        replace=True,
                        n_samples=majority.shape[0], # to match majority class
                        random_state=42) # figure random state out

train_df = pd.concat([majority, new_minority])
train_df = sk.utils.shuffle(train_df, random_state=42)

In [ ]:
train_df.head()

In [ ]:
train_df['bin'].value_counts()

#### Create Training Data

In [ ]:
y_train = train_df['bin']
x_train = train_df.loc[:, train_df.columns != 'bin']

In [ ]:
# fit the model
parameters = {'max_depth': [2, 3, 4, 5, 7],
                'n_estimators': [1, 10, 25, 50, 100],
                'random_state': [42] }


In [ ]:
def perform_grid_search(x_data, y_data):
        rf = sk.ensemble.RandomForestClassifier()

        clf = sk.model_selection.GridSearchCV(rf, parameters, cv=4, scoring='roc_auc', n_jobs=3)

        clf.fit(x_data, y_data)

        print(clf.cv_results_['mean_test_score'])
        print(clf.best_params_['n_estimators'])
        print(clf.best_params_['max_depth'])

        return clf.best_params_['n_estimators'], clf.best_params_['max_depth']

#### Extract Parameters

In [ ]:
n_estimator, depth = perform_grid_search(x_train, y_train)
c_random_state = 42
print('n_estimator: ' + str(n_estimator), '\ndepth: ' + str(depth), '\nc_random_state: ' + str(c_random_state))

#### Refit new model with best params

In [ ]:
rf = sk.ensemble.RandomForestClassifier(max_depth=depth, n_estimators=n_estimator, random_state=c_random_state)
rf.fit(x_train, y_train.values.ravel())

#### Training Metrics

In [ ]:
y_pred_rf = rf.predict_proba(x_train)[:, 1]
y_pred = rf.predict(x_train)
fpr_rf, tpr_rf, _ = sk.metrics.roc_curve(y_train, y_pred_rf)
print('Classification Report')
print(sk.metrics.classification_report(y_train, y_pred))

In [ ]:
print('Confusion Matrix')
print(sk.metrics.confusion_matrix(y_train, y_pred))

In [ ]:
print('Accuracy')
print(sk.metrics.accuracy_score(y_train, y_pred))

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()